In [30]:
# using python 3 to better handle unicode emojis
import emoji
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

%matplotlib inline

# Goal: 
Using   the   training   data   in    ages_train.csv ,   build   a   statistical   model   to   predict   the
age   of   users   in    ages_test.csv,     based   on   any   of   the   information   provided   (profiles, friend   networks,   and   mentions).

# Load data

In [2]:
# load json files: 
ap = pd.read_json('assignment_package/age_profiles.json')
at = pd.read_json('assignment_package/age_tweets.json')
# fp = pd.read_json('assignment_package/friend_profiles.json')
# mp = pd.read_json('assignment_package/mention_profiles.json')

In [3]:
# load csv files:
ages_train = pd.read_csv('assignment_package/ages_train.csv')
mentions = pd.read_csv('assignment_package/mentions.csv')
friends = pd.read_csv('assignment_package/friends.csv')

# Take a look at data

In [4]:
mentions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17695 entries, 0 to 17694
Data columns (total 3 columns):
ID                 17695 non-null int64
MentionedID        17695 non-null int64
MentionedHandle    17695 non-null object
dtypes: int64(2), object(1)
memory usage: 414.8+ KB


In [5]:
mentions.head()

,ID,MentionedID,MentionedHandle
0,612815277,798747864,AlenaBacalia
1,612815277,354590068,hfholden34
2,612815277,401111769,MarilynYousif
3,612815277,35581752,heeeeyerin
4,612815277,1591956090,DogeTheDog


In [6]:
friends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46609 entries, 0 to 46608
Data columns (total 2 columns):
ID          46609 non-null int64
FriendID    46609 non-null int64
dtypes: int64(2)
memory usage: 728.3 KB


In [7]:
friends.head()

,ID,FriendID
0,612815277,574147452
1,612815277,36767299
2,612815277,29374598
3,612815277,67634814
4,612815277,30008146


In [8]:
ages_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1711 entries, 0 to 1710
Data columns (total 2 columns):
ID     1711 non-null int64
Age    1711 non-null int64
dtypes: int64(2)
memory usage: 26.8 KB


In [9]:
ages_train.head()

,ID,Age
0,23605717,21
1,781084760,19
2,102262916,21
3,413710015,21
4,584304916,90


In [10]:
ap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2410 entries, 0 to 2409
Data columns (total 41 columns):
contributors_enabled                  2410 non-null bool
created_at                            2410 non-null datetime64[ns]
default_profile                       2410 non-null bool
default_profile_image                 2410 non-null bool
description                           2410 non-null object
entities                              2410 non-null object
favourites_count                      2410 non-null int64
follow_request_sent                   2410 non-null bool
followers_count                       2410 non-null int64
following                             2410 non-null bool
friends_count                         2410 non-null int64
geo_enabled                           2410 non-null bool
id                                    2410 non-null int64
id_str                                2410 non-null int64
is_translation_enabled                2410 non-null bool
is_translator      

In [11]:
ap.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,id,id_str,is_translation_enabled,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,status,statuses_count,time_zone,url,utc_offset,verified
0,False,2012-06-19 17:54:58,False,False,You'll never find another one like me.,{'description': {'urls': []}},7931,False,274,False,264,True,612815277,612815277,False,False,en,1,,Greggy,False,131516,http://abs.twimg.com/images/themes/theme14/bg.gif,https://abs.twimg.com/images/themes/theme14/bg...,True,https://pbs.twimg.com/profile_banners/61281527...,http://pbs.twimg.com/profile_images/4776662828...,https://pbs.twimg.com/profile_images/477666282...,009999,EEEEEE,EFEFEF,333333,True,False,agregtoremember,"{'retweeted_status': {'retweet_count': 452, 'c...",12230,Atlantic Time (Canada),None,-10800.0,False
1,False,2012-02-12 22:36:17,False,False,"If you like me, i probably like you more. {IG:...",{'description': {'urls': []}},1141,False,481,False,527,True,490791079,490791079,False,False,en,1,9⃣3⃣7⃣,Guff✌️,False,1A1B1F,http://abs.twimg.com/images/themes/theme9/bg.gif,https://abs.twimg.com/images/themes/theme9/bg.gif,False,https://pbs.twimg.com/profile_banners/49079107...,http://pbs.twimg.com/profile_images/5097227545...,https://pbs.twimg.com/profile_images/509722754...,2FC2EF,181A1E,252429,666666,True,False,C_Guffieee,"{'retweet_count': 0, 'created_at': 'Fri Sep 26...",14680,Hawaii,None,-36000.0,False
2,False,2011-03-11 21:44:56,False,False,Success doesn’t come to you…you go to it.,{'description': {'urls': []}},174,False,878,False,953,False,264416315,264416315,False,False,en,2,Wonderland,Starqueria.⚓,False,FCEBB6,http://pbs.twimg.com/profile_background_images...,https://pbs.twimg.com/profile_background_image...,True,https://pbs.twimg.com/profile_banners/26441631...,http://pbs.twimg.com/profile_images/5127311352...,https://pbs.twimg.com/profile_images/512731135...,CE7834,F0A830,78C0A8,5E412F,True,False,_Starqueria,"{'retweeted_status': {'retweet_count': 2614, '...",22857,Hawaii,None,-36000.0,False
3,False,2009-12-17 20:08:55,False,False,NEVER care about what a person says that doesn...,{'description': {'urls': []}},303,False,742,False,701,False,97515432,97515432,False,False,en,30,•Carolina South• 843•,. keeleesha,False,131516,http://pbs.twimg.com/profile_background_images...,https://pbs.twimg.com/profile_background_image...,True,NaN,http://pbs.twimg.com/profile_images/4939742059...,https://pbs.twimg.com/profile_images/493974205...,009999,FFFFFF,EFEFEF,333333,True,False,Keeleesha,"{'retweet_count': 0, 'entities': {'urls': [], ...",71654,Eastern Time (US & Canada),None,-14400.0,False
4,False,2011-03-24 21:14:53,False,False,these niggas some haters & i made myself so ea...,{'description': {'urls': []}},52,False,729,False,688,False,271608323,271608323,False,False,en,0,Seven•five•Seven,killakee :),False,0A0A0A,http://pbs.twimg.com/profile_background_images...,https://pbs.twimg.com/profile_background_image...,True,https://pbs.twimg.com/profile_banners/27160832...,http://pbs.twimg.com/profile_images/4627093362...,https://pbs.twimg.com/profile_images/462709336...,D62D0F,000000,000000,A3A3A3,True,False,KeeLashawn_,"{'retweet_count': 0, 'entities': {'urls': [{'e...",48452,Eastern Time (US & Canada),None,-14400.0,False


In [12]:
at.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28138 entries, 0 to 28137
Data columns (total 25 columns):
contributors                 0 non-null float64
coordinates                  1298 non-null object
created_at                   28138 non-null datetime64[ns]
entities                     28138 non-null object
favorite_count               28138 non-null int64
favorited                    28138 non-null bool
geo                          1298 non-null object
id                           28138 non-null int64
id_str                       28138 non-null int64
in_reply_to_screen_name      5299 non-null object
in_reply_to_status_id        4839 non-null float64
in_reply_to_status_id_str    4839 non-null float64
in_reply_to_user_id          5299 non-null float64
in_reply_to_user_id_str      5299 non-null float64
lang                         28138 non-null object
metadata                     28138 non-null object
place                        1341 non-null object
possibly_sensitive          

In [13]:
at.head()

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,lang,metadata,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,NaN,None,2014-09-26 05:55:45,"{'hashtags': [], 'urls': [], 'symbols': [], 'u...",0,False,None,515379174454747136,515379174454747136,None,NaN,NaN,NaN,NaN,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,7,False,"{'in_reply_to_status_id': None, 'coordinates':...","<a href=""http://twitter.com/download/iphone"" r...",RT @AlenaBacalia: i over think and second gues...,False,"{'follow_request_sent': False, 'profile_image_..."
1,NaN,None,2014-09-26 05:58:17,"{'hashtags': [], 'urls': [], 'symbols': [], 'u...",0,False,None,515379813670854656,515379813670854656,None,NaN,NaN,NaN,NaN,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,80,False,"{'in_reply_to_status_id': None, 'coordinates':...","<a href=""http://twitter.com/download/iphone"" r...",RT @hfholden34: I've wasted too much time on t...,False,"{'follow_request_sent': False, 'profile_image_..."
2,NaN,None,2014-09-26 05:59:47,"{'hashtags': [], 'urls': [], 'symbols': [], 'u...",1,False,None,515380190361288704,515380190361288704,None,NaN,NaN,NaN,NaN,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,1,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",I'm already gone.,False,"{'follow_request_sent': False, 'profile_image_..."
3,NaN,None,2014-09-26 06:00:07,"{'hashtags': [], 'urls': [], 'symbols': [], 'u...",1,False,None,515380273005854721,515380273005854720,None,NaN,NaN,NaN,NaN,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",I'm already something to someone that I don't ...,False,"{'follow_request_sent': False, 'profile_image_..."
4,NaN,None,2014-09-26 08:09:57,"{'hashtags': [], 'urls': [], 'symbols': [], 'u...",1,False,None,515412946608852992,515412946608852992,None,NaN,NaN,NaN,NaN,en,"{'result_type': 'recent', 'iso_language_code':...",None,NaN,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...","I don't know what I'm doing with my life, but ...",False,"{'follow_request_sent': False, 'profile_image_..."


# First Iteration: Use all numeric variables to fit a linear regression model

In [23]:
def extract_emojis(s):
    return ''.join(c for c in str(s) if c in emoji.UNICODE_EMOJI)

def age_profiles_prep(age_profiles, ages_train):
    '''Process age_profiles for modeling'''
    ap_new = pd.concat([age_profiles,
                        pd.DataFrame(age_profiles['status'].apply(pd.Series) \
                        .rename(columns = lambda x: 'status_' + str(x)))], axis=1)
    ap_new['status_emoji'] = ap_new['status_text'].map(lambda x: extract_emojis(x))
    ap_new['emoji_cnt'] = ap_new['status_emoji'].map(lambda x: len(x))
    ap_new = ap_new.set_index('id')
    ages_train = ages_train.set_index('ID')
    age_joined = ap_new.join(ages_train, how='inner')
    return age_joined

In [24]:
age_joined = age_profiles_prep(ap, ages_train)

//anaconda/envs/capstone/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
//anaconda/envs/capstone/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


#### ages_train dataset has 1711 user IDs, age_profiles has 2410 users IDs, after inner join with age_profiles, return 1688 user IDs.

In [26]:
age_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1688 entries, 612815277 to 2385386499
Data columns (total 67 columns):
contributors_enabled                  1688 non-null bool
created_at                            1688 non-null datetime64[ns]
default_profile                       1688 non-null bool
default_profile_image                 1688 non-null bool
description                           1688 non-null object
entities                              1688 non-null object
favourites_count                      1688 non-null int64
follow_request_sent                   1688 non-null bool
followers_count                       1688 non-null int64
following                             1688 non-null bool
friends_count                         1688 non-null int64
geo_enabled                           1688 non-null bool
id_str                                1688 non-null int64
is_translation_enabled                1688 non-null bool
is_translator                         1688 non-null bool
lang  

In [25]:
age_joined.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,id_str,is_translation_enabled,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,status,statuses_count,time_zone,url,utc_offset,verified,status_contributors,status_coordinates,status_created_at,status_entities,status_favorite_count,status_favorited,status_geo,status_id,status_id_str,status_in_reply_to_screen_name,status_in_reply_to_status_id,status_in_reply_to_status_id_str,status_in_reply_to_user_id,status_in_reply_to_user_id_str,status_lang,status_place,status_possibly_sensitive,status_retweet_count,status_retweeted,status_retweeted_status,status_source,status_text,status_truncated,status_0,status_emoji,emoji_cnt,Age
612815277,False,2012-06-19 17:54:58,False,False,You'll never find another one like me.,{'description': {'urls': []}},7931,False,274,False,264,True,612815277,False,False,en,1,,Greggy,False,131516,http://abs.twimg.com/images/themes/theme14/bg.gif,https://abs.twimg.com/images/themes/theme14/bg...,True,https://pbs.twimg.com/profile_banners/61281527...,http://pbs.twimg.com/profile_images/4776662828...,https://pbs.twimg.com/profile_images/477666282...,009999,EEEEEE,EFEFEF,333333,True,False,agregtoremember,"{'retweeted_status': {'retweet_count': 452, 'c...",12230,Atlantic Time (Canada),None,-10800.0,False,NaN,None,Fri Sep 26 16:49:37 +0000 2014,"{'urls': [], 'user_mentions': [{'id': 15919560...",0.0,False,None,5.155437e+17,515543724785684480,None,NaN,None,NaN,None,en,None,NaN,452.0,False,"{'retweet_count': 452, 'created_at': 'Fri Sep ...","<a href=""http://twitter.com/download/android"" ...",RT @DogeTheDog: Wow\n So friday\n ...,False,NaN,,0,21
490791079,False,2012-02-12 22:36:17,False,False,"If you like me, i probably like you more. {IG:...",{'description': {'urls': []}},1141,False,481,False,527,True,490791079,False,False,en,1,9⃣3⃣7⃣,Guff✌️,False,1A1B1F,http://abs.twimg.com/images/themes/theme9/bg.gif,https://abs.twimg.com/images/themes/theme9/bg.gif,False,https://pbs.twimg.com/profile_banners/49079107...,http://pbs.twimg.com/profile_images/5097227545...,https://pbs.twimg.com/profile_images/509722754...,2FC2EF,181A1E,252429,666666,True,False,C_Guffieee,"{'retweet_count': 0, 'created_at': 'Fri Sep 26...",14680,Hawaii,None,-36000.0,False,NaN,None,Fri Sep 26 17:42:07 +0000 2014,"{'urls': [], 'user_mentions': [], 'symbols': [...",0.0,False,None,5.155569e+17,515556938512609280,None,NaN,None,NaN,None,en,None,NaN,0.0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Damn I gotta wait to next week to get the IPho...,False,NaN,😒,1,18
264416315,False,2011-03-11 21:44:56,False,False,Success doesn’t come to you…you go to it.,{'description': {'urls': []}},174,False,878,False,953,False,264416315,False,False,en,2,Wonderland,Starqueria.⚓,False,FCEBB6,http://pbs.twimg.com/profile_background_images...,https://pbs.twimg.com/profile_background_image...,True,https://pbs.twimg.com/profile_banners/26441631...,http://pbs.twimg.com/profile_images/5127311352...,https://pbs.twimg.com/profile_images/512731135...,CE7834,F0A830,78C0A8,5E412F,True,False,_Starqueria,"{'retweeted_status': {'retweet_count': 2614, '...",22857,Hawaii,None,-36000.0,False,NaN,None,Fri Sep 26 02:57:32 +0000 2014,"{'urls': [], 'media': [{'media_url': 'http://p...",0.0,False,None,5.153343e+17,515334328113307649,None,NaN,None,NaN,None,en,None,False,2614.0,False,"{'retweet_count': 2614, 'entities': {'urls': [...","<a href=""http://twitter.com/download/android"" ...",RT @ImNeverChillin: How your girl look at you ...,False,NaN,,0,18
97515432,False,2009-12-17 20:08:5